In [2]:
import hail

import hailtop.batch as hb

backend = hb.ServiceBackend(billing_project = 'finucane-splicing-hj',
                            remote_tmpdir = 'gs://sqtl_gtex')

import pandas as pd

Loading BokehJS ...

Explaination of script last updates 2 17 2024
--
This script creates a hail batch object that is able to send a job to a hail cluster. 

The jobs that we will be sending is the empirical bayes pipeline. 





run on simulations
--

In [2]:
b = hb.Batch(backend=backend, name='emp_bayes')
num_jobs = 25 #samples every 100 index



#scale to all lists of simulations
simulation_type=['single_1',
                'single_2',
                'double_1','double_2',
                 'triple_1', 'triple_2']
data_amt=['low', 'mid', 'high']
i=0
k=0
for i in range(len(simulation_type)):

    for k in range(len(data_amt)):
        l = b.new_job(name='all_data_ammounts')
        l.memory('20Gi')
        l.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v4')
        tissue='test_tissue'
    #localizing the files
        input_script = "/home/run_optimizer_alt_ss_w_mixed_effects.py"    
        
        input_file_tissue = b.read_input("gs://sqtl_gtex/files/testing/mixture_testing/inputs_simulations/"+data_amt[k]+"_NVE_"+simulation_type[i]+"_counts.csv.gz")
        
        input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/testing/mixture_testing/inputs_simulations/filtering_exons_in_test_sample.csv.gz")
            
        
        for job_index in range(num_jobs):
            
            j = b.new_job(name='run_emp_bayes_test_')
            j.memory('20Gi')
            j.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v4')
        
            j.command(f'python3 {input_script} \
                            --tissue {tissue} \
                                --filter_file {input_file_to_filter_events} \
                                --tissue_file {input_file_tissue} \
                                --output_file {j.output} \
                                --num_jobs {num_jobs} \
                                --job_index {job_index}')
        
            b.write_output(j.output, f'gs://sqtl_gtex/files/testing/mixture_testing/outputs_simulations_new_pipeline/{data_amt[k]}_{simulation_type[i]}_EF_FPR_5_alphas_betas-{job_index}.csv')
        
            
b.run()



    

Output()

Output()


KeyboardInterrupt
]8;id=306908;https://batch.hail.is/batches/8241673\8241673]8;;\ ━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  15%  68/468 0:03:22 0:05:11


]8;id=532779;https://batch.hail.is/batches/8241673\8241673]8;;\ ━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  15%  68/468 0:03:22 0:05:11

In [22]:
b = hb.Batch(backend=backend, name='emp_bayes')
directory='gs://sqtl_gtex/files/testing/mixture_testing/outputs_simulations_new_pipeline/'

j = b.new_job(name='run_emp_bayes_test_')
j.memory('20Gi')
j.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v4')
for i in range(len(simulation_type)):
    for k in range(len(data_amt)):
        files_given_sim=[]
        
        for job_index in range(num_jobs):
    
            file=data_amt[k]+'_'+simulation_type[i]+'_EF_FPR_5_alphas_betas-'+str(job_index)+'.csv'
            
            o_file=b.read_input(directory+file)

            files_given_sim.append(o_file)
            new_file_name=data_amt[k]+'_'+simulation_type[i]+'_EF_FPR_5_alphas_betas.csv'
        print(files_given_sim[0:10])
        ofile = hb.concatenate(b, files_given_sim, branching_factor=2)
        b.write_output(ofile, f'gs://sqtl_gtex/files/testing/mixture_testing/outputs_simulations_new_pipeline/all_{new_file_name}') 
b.run()

[__RESOURCE_FILE__1345, __RESOURCE_FILE__1346, __RESOURCE_FILE__1347, __RESOURCE_FILE__1348, __RESOURCE_FILE__1349, __RESOURCE_FILE__1350, __RESOURCE_FILE__1351, __RESOURCE_FILE__1352, __RESOURCE_FILE__1353, __RESOURCE_FILE__1354]
[__RESOURCE_FILE__1397, __RESOURCE_FILE__1398, __RESOURCE_FILE__1399, __RESOURCE_FILE__1400, __RESOURCE_FILE__1401, __RESOURCE_FILE__1402, __RESOURCE_FILE__1403, __RESOURCE_FILE__1404, __RESOURCE_FILE__1405, __RESOURCE_FILE__1406]
[__RESOURCE_FILE__1449, __RESOURCE_FILE__1450, __RESOURCE_FILE__1451, __RESOURCE_FILE__1452, __RESOURCE_FILE__1453, __RESOURCE_FILE__1454, __RESOURCE_FILE__1455, __RESOURCE_FILE__1456, __RESOURCE_FILE__1457, __RESOURCE_FILE__1458]
[__RESOURCE_FILE__1501, __RESOURCE_FILE__1502, __RESOURCE_FILE__1503, __RESOURCE_FILE__1504, __RESOURCE_FILE__1505, __RESOURCE_FILE__1506, __RESOURCE_FILE__1507, __RESOURCE_FILE__1508, __RESOURCE_FILE__1509, __RESOURCE_FILE__1510]
[__RESOURCE_FILE__1553, __RESOURCE_FILE__1554, __RESOURCE_FILE__1555, __RESO

Output()

Output()

batch 8241677 complete: success


# run new pipeline on all tissues




alt ss
--

In [4]:
data_directory='/Users/hannahjacobs/Dropbox (MIT)/GradSchool/Finucane/splicing_variation_in_humans_2022/data/00_data/emp_bayes_input/'


alt_splice_sites=pd.read_csv(data_directory+'alt_splice_sites_filtered_for_popPSI.csv.gz', compression='gzip')





gtex_tissue_list=list(set(alt_splice_sites.tissue))

In [ ]:
b = hb.Batch(backend=backend, name='emp_bayes')
num_jobs = 1000

#list_of_tissues=['Whole_Blood','Adipose_Subcutaneous']

list_of_tissues=gtex_tissue_list

b = hb.Batch(backend=backend, name='emp_bayes')

for tissue in list_of_tissues:
    
    
  
    for job_index in range(num_jobs):
    
        j = b.new_job(name='run_emp_bayes_'+tissue)
        j.memory('20Gi')
       
        #localizing the files
        
        j.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v4')
            
        input_script = "/home/run_optimizer_alt_ss_w_mixed_effects.py"

        input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')
    
        input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/alt_ss_for_EF_algorithm.csv.gz")
        
        j.command(f'python3 {input_script} \
                            --tissue {tissue} \
                                --filter_file {input_file_to_filter_events} \
                                --tissue_file {input_file_tissue} \
                                --output_file {j.output} \
                                --num_jobs {num_jobs} \
                                --job_index {job_index}')
        
        b.write_output(j.output, 'gs://sqtl_gtex/files/testing/mixture_testing/real_data_outputs/' + tissue + '_alt_ss_EF_5_FPR_5_' + str(job_index) + '_alphas_betas.csv')

      
b.run() 
    

Output()

Output()

Output()

Output()

all alt ss combine files
--




In [6]:
b = hb.Batch(backend=backend, name='emp_bayes')
directory='gs://sqtl_gtex/files/testing/mixture_testing/real_data_outputs/'
num_jobs=1000
j = b.new_job(name='run_emp_bayes_test_')
j.memory('20Gi')
j.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v4')
list_of_tissues=gtex_tissue_list
for tissue in gtex_tissue_list:
    for job_index in range(num_jobs):

        files_given_sim=[]
            

        
        file=tissue + '_alt_ss_EF_5_FPR_5_' + str(job_index) + '_alphas_betas.csv'
                
        o_file=b.read_input(directory+file)
    
        files_given_sim.append(o_file)
        new_file_name = tissue+'_EF_FPR_5_alphas_betas.csv'
 
        ofile = hb.concatenate(b, files_given_sim, branching_factor=2)
        b.write_output(ofile, f'gs://sqtl_gtex/files/testing/mixture_testing/real_data_outputs/all_{new_file_name}') 
b.run()

Output()

Output()

batch 8242118 complete: success


test skipped exons
--

In [17]:
data_dir='/Users/hannahjacobs/MIT Dropbox/Hannah Jacobs/GradSchool/Finucane/splicing_variation_in_humans_2022/data/'

In [19]:
skipped_exons_w_gene_names_and_assigned_abc_introns=pd.read_csv(data_dir+'00_data/skipped_exons_w_gene_names_and_assigned_abc_introns.csv.gz', compression='gzip',low_memory=False)

In [27]:
b = hb.Batch(backend=backend, name='emp_bayes')
num_jobs = 1000

#list_of_tissues=['Whole_Blood','Adipose_Subcutaneous']

list_of_tissues=['Adipose_Subcutaneous','Whole_Blood']

b = hb.Batch(backend=backend, name='emp_bayes')

for tissue in list_of_tissues:
    #range(num_jobs)
    for job_index in range(10):
    
        j = b.new_job(name='run_emp_bayes_'+tissue)
        j.memory('20Gi')
       
        #localizing the files
        
        j.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v4')
            
        input_script = "/home/run_optimizer_SE_w_mixed_effects.py"
    
        input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')
    
        input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/skipped_exons_w_gene_names_and_assigned_abc_introns.csv.gz")
       
        j.command(f'python3 {input_script} \
                            --tissue {tissue} \
                                --filter_file {input_file_to_filter_events} \
                                --tissue_file {input_file_tissue} \
                                --output_file {j.output} \
                                --num_jobs {num_jobs} \
                                --job_index {job_index}')
        
        b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/mixture_model/SE/optimizer_results/' + tissue + '_SE_EF_5_FPR_5_' + str(job_index) +'alphas_betas.csv')
#  b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/mixture_model/alt_ss/optimizer_results/' + tissue + '_alt_ss_EF_5_FPR_5_' + str(job_index) +'_alphas_betas.csv')
    
b.run() 
    

Output()

Output()

batch 8242069 complete: success


run on all skipped exons
--

In [ ]:

num_jobs = 1000


b = hb.Batch(backend=backend, name='emp_bayes')

for tissue in gtex_tissue_list:
    #range(num_jobs)
    
    for job_index in range(num_jobs):
        j = b.new_job(name='run_emp_bayes_'+tissue)
        j.memory('20Gi')
           
            #localizing the files
            
        j.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v4')
                
        input_script = "/home/run_optimizer_SE_w_mixed_effects.py"
        
        input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')
        
        input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/skipped_exons_w_gene_names_and_assigned_abc_introns.csv.gz")
           
        j.command(f'python3 {input_script} \
                                --tissue {tissue} \
                                    --filter_file {input_file_to_filter_events} \
                                    --tissue_file {input_file_tissue} \
                                    --output_file {j.output} \
                                    --num_jobs {num_jobs} \
                                    --job_index {job_index}')
            
        b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/mixture_model/SE/optimizer_results/' + tissue + '_SE_EF_5_FPR_5_' + str(job_index) +'_alphas_betas.csv')
     
b.run() 
    

Output()

Output()

Output()

Output()

merge the files into one
--


In [ ]:
file_dir='gs://sqtl_gtex/files/EF_outputs/mixture_model/SE/optimizer_results/'
+ tissue + '_SE_EF_5_FPR_5_' + str(job_index) +'alphas_betas.csv'
tissue=tissue
j.command(f'{file_dir}'+tissue+'_SE_EF_5_FPR_5_*alphas_betas'+)
        


other code
--

In [29]:
b = hb.Batch(backend=backend, name='emp_bayes')
num_jobs = 500 

j.memory('20Gi')

tissue=['Esophagus_Mucosa']

b = hb.Batch(backend=backend, name='emp_bayes')
for tissue in ['Esophagus_Mucosa']:
    
    
    for job_index in range(5):
        j = b.new_job(name='run_emp_bayes_'+tissue)
        j.memory('20Gi')
       
        #localizing the files
        
        j.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v4')
            
        input_script = "/home/run_optimizer_SE_w_mixed_effects.py"
    
        input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')
    
        input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/skipped_exons_w_gene_names_and_assigned_abc_introns.csv.gz")
        
        j.command(f'python3 {input_script} \
                            --tissue {tissue} \
                                --filter_file {input_file_to_filter_events} \
                                --tissue_file {input_file_tissue} \
                                --output_file {j.output} \
                                --num_jobs {num_jobs} \
                                --job_index {job_index}')
    
        b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/mixture_model/SE/optimizer_results/' + tissue + '_SE_EF_5_FPR_5_' + str(job_index) +'alphas_betas.csv')
    b.run() 

Output()

Output()

batch 8236401 complete: failure


In [7]:
output_directory='/Users/hannahjacobs/MIT Dropbox/Hannah Jacobs/GradSchool/Finucane/splicing_variation_in_humans_2022/scripts/01_analysis_pipeline/00_run_emprical_bayes/emp_bayes_in_hail_pipeline-testing/data/mixture_implementation_testing/'

import pandas as pd

In [4]:
import numpy as np

In [2]:


whole_blood = pd.read_csv('/Users/hannahjacobs/MIT Dropbox/Hannah Jacobs/GradSchool/Finucane/splicing_variation_in_humans_2022/GTEX sQTLs/GTEx_Analysis_v8_sQTL_leafcutter_counts/Whole_Blood_perind_numers.counts.gz',
                         low_memory=False,compression='gzip',
                          sep=' ', index_col=0)



In [4]:
b = hb.Batch(backend=backend, name='emp_bayes_alt_ss')
for tissue in gtex_tissue_list:
    
    
    
    j = b.new_job(name='run_emp_bayes_'+tissue)
    j.memory('10Gi')
   
    #localizing the files
    

    
    
        
    input_script = "/home/run_optimizer_alt_ss.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')

    input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/alt_ss_for_EF_algorithm.csv.gz")
    
    j.image('us-central1-docker.pkg.dev/finucane-splicing-hj/testing/testing_beta_binom:v1')

    j.command(f'python3 {input_script} \
                --tissue {tissue} \
                --filter_file {input_file_to_filter_events} \
                --tissue_file {input_file_tissue} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/testing/EF_ouputs/alt_ss/optimizer_results/removed_prior/'+tissue+'_alt_ss_EF_5_FPR_5_alphas_betas.csv')

b.run() 

/opt/anaconda3/envs/running_hail_env/lib/python3.11/site-packages/rich/live.py:229: UserWarning:

install "ipywidgets" for Jupyter support

batch 8225675 complete: success


In [6]:
b = hb.Batch(backend=backend, name='assign_EF_alt_ss')
for tissue in gtex_tissue_list:
    
    j = b.new_job(name='assign_EF_'+tissue)
    j.memory('7Gi')
   
    #localizing the files
    
    input_script = "/home/assign_EF.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/files/EF_outputs/alt_ss/optimizer_results/"+tissue+"_alt_ss_alphas_betas.csv")

    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 {input_script} \
                --tissue_file {input_file_tissue} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/alt_ss/assigned_EFs/'+tissue+'_EFs_alt_ss.csv')

b.run() 

Output()

batch 7689646 complete: success


In [ ]:
b = hb.Batch(backend=backend, name='concat_EFs')
for tissue in gtex_tissue_list:
    
    j = b.new_job(name='concat_EFs'+tissue)
    j.memory('7Gi')
   
    #localizing the files
    

    input_file_tissue = b.read_input("gs://sqtl_gtex/files/EF_outputs/alt_ss/assigned_EFs/"+tissue+"_EFs_alt_ss.csv")

    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 dfs = list() \
            for i, f in enumerate{tissue}: \
                data = pd.read_csv({tissue_file}) \
                data=data.assign(tissue={tissue}) \
                dfs.append(data)

                df = pd.concat(dfs, ignore_index=True)
                
                df.to_csv({j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/alt_ss/all_splicing_events.csv.gz')



In [37]:
b = hb.Batch(backend=backend, name='assign_EF_SE')
for tissue in gtex_tissue_list:
    
    j = b.new_job(name='assign_EF_'+tissue)
    j.memory('10Gi')
   
    #localizing the files
    
    input_script = "/home/assign_EF.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/files/EF_outputs/skipped_exons/optimizer_results/"+tissue+"_SE_alphas_betas.csv")

    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 {input_script} \
                --tissue_file {input_file_tissue} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/skipped_exons/assigned_EFs/'+tissue+'_EFs_SE.csv')

b.run() 

Output()

batch 8101537 complete: success


 0. test/debug with a single tissue


In [5]:
b = hb.Batch(backend=backend, name='emp_bayes')
for tissue in ['Whole_Blood']:
    
    
    
    j = b.new_job(name='run_emp_bayes_'+tissue)
    j.memory('7Gi')
   
    #localizing the files
    

    
    
    
    
    input_script = "/home/run_EF.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')

    input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/alt_ss_for_EF_algorithm.csv.gz")
    
    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 {input_script} \
                --tissue {tissue} \
                --filter_file {input_file_to_filter_events} \
                --tissue_file {input_file_tissue} \
                --lower_bound {5} \
                --FPR {0.05} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/alt_ss/EF_5_percent_FPR_5_percent/'+tissue+'_alt_ss_EF_5_FPR_5_values.csv')

b.run() 
    

Output()


KeyboardInterrupt



1. Run across all tissues

In [7]:
data_directory='/Users/hnjacobs/Dropbox (MIT)/GradSchool/Finuance/splicing_variation_in_humans_2022/data/00_data/emp_bayes_input/'


alt_splice_sites=pd.read_csv(data_directory+'alt_splice_sites_filtered_for_popPSI.csv.gz', compression='gzip')





gtex_tissue_list=list(set(alt_splice_sites.tissue))

gtex_tissue_list

In [17]:
#psi_directory='/Users/hnjacobs/Dropbox (MIT)/GradSchool/Finuance/GTEx sQTLs/GTEx_Analysis_v8_sQTL_leafcutter_counts/'


run on all tissues EF 5%
-

In [21]:
b = hb.Batch(backend=backend, name='emp_bayes')
for tissue in gtex_tissue_list:
    
    
    
    j = b.new_job(name='run_emp_bayes_'+tissue)
    j.memory('7Gi')
   
    #localizing the files
    

    
    
    
    
    input_script = "/home/run_EF.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')

    input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/alt_ss_for_EF_algorithm.csv.gz")
    
    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 {input_script} \
                --tissue {tissue} \
                --filter_file {input_file_to_filter_events} \
                --tissue_file {input_file_tissue} \
                --lower_bound {0.05} \
                --FPR {0.05} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/alt_ss/EF_5_percent_FPR_5_percent/'+tissue+'_alt_ss_EF_5_FPR_5_values.csv')

b.run() 
    

Output()

A transient error occured. We will automatically retry. Do not be alarmed. We have thus far seen 2 transient errors
(current delay: 0.2). The most recent error was <class 'asyncio.exceptions.TimeoutError'> .

batch 7597050 complete: cancelled


In [8]:
b = hb.Batch(backend=backend, name='emp_bayes')
for tissue in gtex_tissue_list:
    
    
    
    j = b.new_job(name='run_emp_bayes_'+tissue)
    j.memory('7Gi')
   
    #localizing the files
    

    
    
    
    
    input_script = "/home/run_EF.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')

    input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/alt_ss_for_EF_algorithm.csv.gz")
    
    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 {input_script} \
                --tissue {tissue} \
                --filter_file {input_file_to_filter_events} \
                --tissue_file {input_file_tissue} \
                --lower_bound {0.20} \
                --FPR {0.05} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/alt_ss/EF_20_percent/'+tissue+'_alt_ss_EF_20_FPR_5_values.csv')

b.run() 
    

Output()

batch 7624031 complete: cancelled


In [ ]:
python3 run_EF.py \
                --tissue 'Adipose_Subcutaneous' \
                --tissue_file '/Users/hnjacobs/Dropbox (MIT)/GradSchool/Finuance/GTEX sQTLs/GTEx_Analysis_v8_sQTL_leafcutter_counts/Adipose_Subcutaneous_perind_numers.counts.gz' \
                --filter_file '/Users/hnjacobs/Dropbox (MIT)//GradSchool/Finuance/splicing_variation_in_humans_2022/data/alt_splice_sites_filtered_for_popPSI.csv.gz' \
                --output_file Adipose_output.csv.gz
    
    

In [ ]:
b = (backend=backend, name='emp_bayes')
for tissue in ['Adipose_Subcutaneous']:
    
    
    
    j = b.new_job(name='run_emp_bayes_'+tissue)
    j.memory('7Gi')
   
    #localizing the files
    

    
    
    
    
    input_script = "/home/run_EF.py"

    input_file_tissue = b.read_input("gs://sqtl_gtex/exon_exon_counts_gtex/"+tissue+'_perind_numers.counts.gz')

    input_file_to_filter_events = b.read_input("gs://sqtl_gtex/files/EF_inputs/alt_splice_sites_filtered_for_popPSI.csv.gz")

    j.image('gcr.io/finucane-splicing-hj/python-11-image-hj:v1')

    j.command(f'python3 {input_script} \
                --tissue {tissue} \
                --filter_file {input_file_to_filter_events} \
                --tissue_file {input_file_tissue} \
                --output_file {j.output}')

    b.write_output(j.output, 'gs://sqtl_gtex/files/EF_outputs/'+tissue+'_EF_values.csv')

b.run() 
    